In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp
from numpy import pi, log, exp, real

from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix, eye, csc_matrix
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.interpolate import RegularGridInterpolator

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class hedged_portfolio:
    def __init__(self, simulated_prices, S0, K, T, r, args, option_type, model_type):
        self.S = simulated_prices  # shape: (M, N)
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.args = args
        self.option_type = option_type  # 'call' or 'put'
        self.model_type = model_type    # e.g., 'BSM'

    def payoff(self, ST):
        if self.option_type == 'call':
            return np.maximum(ST - self.K, 0)
        else:
            return np.maximum(self.K - ST, 0)

    def calculate_portfolio(self):
        M, N = self.S.shape
        ST = self.S[:, -1]
        V_t = self.payoff(ST)
        eps = 0.01 * self.S0

        # Backward loop for replicating portfolio
        phi_all = np.zeros((M, N-1))  # stock holdings
        psi_all = np.zeros((M, N-1))  # no relevance here

        V_all = np.zeros((M, N))
        V_all[:, -1] = V_t

        pf_all = np.zeros((M, N)) #Not same as option price in delta hedged portfolio

        dt = self.T/N
        discount = np.exp(-self.r * dt)

        # compounding * pf_t = phi_t * S_t - V_t => V_t * discount = phi_t * S_t * discount - pf_t
        #riskless portfolio
        for t in reversed(range(N-1)):
            X = self.S[:, t].reshape(-1, 1)
            X = X * discount
            y = V_t * discount  # discount option value to time t

            # Linear regression: V or pf = phi * S_t + psi
            model = LinearRegression().fit(X, y)
            phi_t = model.coef_[0]
            pf_t = model.intercept_

            V_t = model.predict(X)

            phi_all[:, t] = -1 * phi_t
            pf_all[:, t] = pf_t
            V_all[:, t] = V_t

        return phi_all, psi_all, pf_all, V_all

    def plot_portfolio(self, phi_all, psi_all, pf_all, V_all):
        M, N = self.S.shape

        # Time grids
        time_grid = np.linspace(0, self.T, N)
        time_grid_minus = np.linspace(0, self.T, N-1)

        # Averages and std devs over paths
        mean_S = np.mean(self.S, axis=0)
        std_S = np.std(self.S, axis=0)

        mean_pf = np.mean(pf_all, axis=0)
        std_pf = np.std(pf_all, axis=0)

        mean_phi = np.mean(phi_all, axis=0)
        std_phi = np.std(phi_all, axis=0)

        mean_V = np.mean(V_all, axis=0)
        std_V = np.std(V_all, axis=0)

        # P&L from dynamic hedging
        pnl_all = pf_all - pf_all[:, 0].reshape(-1, 1)
        mean_pnl = np.mean(pnl_all, axis=0)
        std_pnl = np.std(pnl_all, axis=0)

        # Hedging error: Π - V
        hedging_error = pf_all - V_all
        mean_err = np.mean(hedging_error, axis=0)
        std_err = np.std(hedging_error, axis=0)

        # === Subplots ===
        fig, axs = plt.subplots(6, 1, figsize=(10, 20), sharex=True)

        # 1. Stock Price
        axs[0].plot(time_grid, mean_S, color='blue', lw=2, label='Mean Stock Price')
        axs[0].fill_between(time_grid, mean_S - std_S, mean_S + std_S, alpha=0.2, color='blue', label='±1σ Band')
        axs[0].axhline(mean_S[-1], color='red', linestyle='--', label=f"Final = {mean_S[-1]:.2f}")
        axs[0].set_ylabel("Stock Price")
        axs[0].set_title("Mean Stock Price with ±1σ")
        axs[0].legend()
        axs[0].grid(True)
        #axs[0].invert_xaxis()

        # 2. Option value from regression
        axs[1].plot(time_grid, mean_V, color='purple', lw=2, label='Mean option value')
        axs[1].fill_between(time_grid, mean_V - std_V, mean_V + std_V, alpha=0.2, color='purple', label='±1σ Band')
        axs[1].axhline(mean_V[-1], color='red', linestyle='--', label=f"Final = {mean_V[-1]:.2f}")
        axs[1].set_ylabel("Option value")
        axs[1].set_title("Mean option alue (regressand) with ±1σ")
        axs[1].legend()
        axs[1].grid(True)
        #axs[1].invert_xaxis()

        # 3. Delta or phi from Regression
        axs[2].plot(time_grid_minus, mean_phi, color='green', lw=2, label='Mean Delta Value')
        axs[2].fill_between(time_grid_minus, mean_phi - std_phi, mean_phi + std_phi, alpha=0.2, color='green', label='±1σ Band')
        axs[2].axhline(mean_phi[-1], color='red', linestyle='--', label=f"Final = {mean_phi[-1]:.2f}")
        axs[2].set_ylabel("Delta Value")
        axs[2].set_title("Mean Delta Value (regressor) with ±1σ")
        axs[2].legend()
        axs[2].grid(True)
        #axs[1].invert_xaxis()

        # 4. Portfolio value from regression
        axs[3].plot(time_grid, mean_pf, color='purple', lw=2, label='Mean hedged portfolio value')
        axs[3].fill_between(time_grid, mean_pf - std_pf, mean_pf + std_pf, alpha=0.2, color='purple', label='±1σ Band')
        axs[3].axhline(mean_pf[-1], color='red', linestyle='--', label=f"Final = {mean_pf[-1]:.2f}")
        axs[3].set_ylabel("portfolio value")
        axs[3].set_title("Mean hedged portfolio option alue (regressor) with ±1σ")
        axs[3].legend()
        axs[3].grid(True)
        #axs[3].invert_xaxis()

        # 5. P&L from Dynamic Rebalancing
        axs[4].plot(time_grid, mean_pnl, color='brown', lw=2, label='Mean P&L from Hedging')
        axs[4].fill_between(time_grid, mean_pnl - std_pnl, mean_pnl + std_pnl, alpha=0.2, color='brown', label='±1σ Band')
        axs[4].axhline(mean_pnl[-1], color='red', linestyle='--', label=f"Final P&L ≈ {mean_pnl[-1]:.2f}")
        axs[4].set_xlabel("Time")
        axs[4].set_ylabel("PnL (Π - Π₀)")
        axs[4].set_title("Hedging P&L from Dynamic Rebalancing")
        axs[4].legend()
        axs[4].grid(True)
        #axs[4].invert_xaxis()

        # 6. Hedging Error: Π - V
        axs[5].plot(time_grid, mean_err, color='orange', lw=2, label='Mean Hedging Error (Π − V)')
        axs[5].fill_between(time_grid, mean_err - std_err, mean_err + std_err, alpha=0.2, color='orange', label='±1σ Band')
        axs[5].axhline(0, color='black', linestyle='--', label=f"Final = {mean_err[-1]:.2f}")
        axs[5].set_ylabel("Error")
        axs[5].set_title("Mean Hedging Error Over Time,  option value from (Model - regression)")
        axs[5].legend()
        axs[5].grid(True)
        #axs[5].invert_xaxis()

        plt.tight_layout()
        plt.show()
